# **Test case #1**: Searching and booking accomodation #

**Situation**: Guest Alice Johnson (id 25) is seeking to find a property that can accommodate 4 people and a dog, with at least 2 bedrooms, available for a stay from the 4th to the 8th of August.

In [ ]:
SELECT p.property_id, p.title, p.description, p.price_per_night, c.currency_code AS currency,
       a.city,
       co.country_name AS country,
       AVG(r.rating) AS average_rating
FROM properties AS p
JOIN currencies AS c ON p.currency_payment = c.currency_code
JOIN addresses AS a ON p.address_id = a.address_id
JOIN countries AS co ON a.country = co.country_code
LEFT JOIN bookings AS b ON p.property_id = b.property_id
LEFT JOIN reviews AS r ON b.booking_id = r.booking_id
WHERE p.beds >= 4
  AND p.is_active = TRUE
  AND p.pets_allowed = TRUE
  AND p.property_id NOT IN (SELECT pa.property_id
                            FROM property_availability AS pa
                            WHERE (pa.starts_date <= '2024-08-04' AND pa.end_date > '2024-08-04')
                               OR (pa.starts_date < '2024-08-08' AND pa.end_date >= '2024-08-08')
                               OR (pa.starts_date >= '2024-08-04' AND pa.end_date <= '2024-08-08'))
GROUP BY p.property_id;

In [ ]:
INSERT INTO bookings (property_id, user_id, booking_date, checkin_date, checkout_date, total_price) VALUES
(11, 25, '2024-04-25 19:32:48', '2024-08-04', '2024-08-04', 720.00);

In [ ]:
INSERT INTO financial_transactions (user_id, booking_id, transaction_type, amount, currency, transaction_date, status) VALUES
(25, 41, 'Hold', 720.00, 'GBP', '2024-04-25 19:32:58', 'Pending Payment from Guest');

In [ ]:
INSERT INTO property_availability (property_id, starts_date, end_date, reason) VALUES
(11, '2024-08-04', '2024-08-08', 'Booked');

# **Test #2**: Searching accomodation with specific equipment #

**Situation**: A guest is seeking to find a property with a pool in the US.

In [ ]:
SELECT p.property_id, p.title, p.description, p.price_per_night, c.currency_code AS currency,
       ad.city,
       co.country_name AS country,
       AVG(r.rating) AS average_rating
FROM properties AS p
JOIN currencies AS c ON p.currency_payment = c.currency_code
JOIN addresses AS ad ON p.address_id = ad.address_id
JOIN countries AS co ON ad.country = co.country_code
JOIN property_amenities AS pa ON p.property_id = pa.property_id
JOIN amenities AS am ON pa.amenity_id = am.amenity_id
LEFT JOIN bookings AS b ON p.property_id = b.property_id
LEFT JOIN reviews AS r ON b.booking_id = r.booking_id
WHERE am.amenity_id = 9
    AND co.country_code = 'US'
GROUP BY p.property_id;

# **Test #3**: Investigating user’s refund request #

**Situation**: A guest, Jane Smith (id 24) contacted the support after her stay (booking id 2) in a property to request a refund. Customer service representative must pull out the details related to the booking

In [ ]:
SELECT us.user_id, us.first_name, us.last_name, us.email,
       c.dialling_code, us.phone_number,
       b.booking_id, b.booking_date, checkin_date, checkout_date,
       h.user_id AS host_id,
       ft.transaction_type, ft.amount, ft.currency, ft.transaction_date, ft.status
FROM users AS us
JOIN bookings AS b ON us.user_id = b.user_id
JOIN properties AS p ON b.property_id = p.property_id
JOIN hosts AS h ON p.user_id = h.user_id
JOIN addresses AS a ON us.address_id = a.address_id
JOIN countries AS c ON a.country = c.country_code
JOIN financial_transactions AS ft ON b.booking_id = ft.booking_id
WHERE us.user_id = 24
  AND b.booking_id = 2;